In [1]:
import os
import shutil
import random
import cv2
import numpy as np
from tqdm import tqdm
from imgaug import augmenters as iaa

# INPUT: your resized dataset
input_dir = "Resized_IMG_CLASSES"
train_dir = "Split_IMG_CLASSES/train"
test_dir = "Split_IMG_CLASSES/test"
classes_to_augment = ["1. Eczema", "3. Atopic Dermatitis", "6. Fungal Infections", "7. Viral Infections"]
target_augmented_count = 1500
test_split_ratio = 0.2

# Mild augmentation sequence
augmenter = iaa.Sequential([
    iaa.Fliplr(0.3),
    iaa.LinearContrast((0.9, 1.1)),
    iaa.AdditiveGaussianNoise(scale=(0, 0.02 * 255))
])

# Prepare folders
for base in [train_dir, test_dir]:
    os.makedirs(base, exist_ok=True)

# Split and optionally augment
for class_name in os.listdir(input_dir):
    class_path = os.path.join(input_dir, class_name)
    if not os.path.isdir(class_path):
        continue

    images = [img for img in os.listdir(class_path) if img.lower().endswith(('.jpg', '.png', '.jpeg'))]
    random.shuffle(images)
    split_idx = int(len(images) * (1 - test_split_ratio))

    train_images = images[:split_idx]
    test_images = images[split_idx:]

    # Create train/test subfolders
    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)

    # Copy test images
    for img in test_images:
        src = os.path.join(class_path, img)
        dst = os.path.join(test_dir, class_name, img)
        shutil.copy2(src, dst)

    # Copy and augment training images
    for img in train_images:
        src = os.path.join(class_path, img)
        dst = os.path.join(train_dir, class_name, img)
        shutil.copy2(src, dst)

    if class_name in classes_to_augment:
        print(f"\n🔁 Augmenting class: {class_name}")
        current_images = os.listdir(os.path.join(train_dir, class_name))
        needed = target_augmented_count - len(current_images)
        if needed > 0:
            idx = 0
            while len(os.listdir(os.path.join(train_dir, class_name))) < target_augmented_count:
                img_name = train_images[idx % len(train_images)]
                img_path = os.path.join(class_path, img_name)
                image = cv2.imread(img_path)

                if image is None:
                    idx += 1
                    continue

                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                aug_img = augmenter(image=image)
                aug_img = cv2.cvtColor(aug_img, cv2.COLOR_RGB2BGR)

                aug_name = f"aug_{idx}_{img_name}"
                aug_path = os.path.join(train_dir, class_name, aug_name)
                cv2.imwrite(aug_path, aug_img)

                idx += 1

print("\nTrain/Test split complete.")
print(f" Training set in: {train_dir}")
print(f" Testing set in: {test_dir}")



🔁 Augmenting class: 1. Eczema

🔁 Augmenting class: 3. Atopic Dermatitis

🔁 Augmenting class: 6. Fungal Infections

🔁 Augmenting class: 7. Viral Infections

Train/Test split complete.
 Training set in: Split_IMG_CLASSES/train
 Testing set in: Split_IMG_CLASSES/test


In [3]:
import os
import numpy as np
import cv2
from tqdm import tqdm
from skimage.feature import local_binary_pattern, graycomatrix, graycoprops
from skimage.color import rgb2gray
from sklearn.preprocessing import StandardScaler

def extract_lbp(image):
    gray = rgb2gray(image)
    lbp = local_binary_pattern((gray * 255).astype(np.uint8), P=8, R=1, method='uniform')
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 11), range=(0, 10))
    return hist.astype(np.float32)

def extract_glcm(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    glcm = graycomatrix(gray, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    dissimilarity = graycoprops(glcm, 'dissimilarity')[0, 0]
    homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
    energy = graycoprops(glcm, 'energy')[0, 0]
    return np.array([contrast, dissimilarity, homogeneity, energy])

def extract_manual_features_from_folder(base_dir):
    features = []
    labels = []
    class_names = sorted(os.listdir(base_dir))
    for label_index, class_name in enumerate(class_names):
        class_path = os.path.join(base_dir, class_name)
        for img_name in tqdm(os.listdir(class_path), desc=class_name):
            img_path = os.path.join(class_path, img_name)
            image = cv2.imread(img_path)
            if image is None:
                continue
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (224, 224))
            lbp_feat = extract_lbp(image)
            glcm_feat = extract_glcm(image)
            combined = np.hstack([lbp_feat, glcm_feat])
            features.append(combined)
            labels.append(label_index)
    return np.array(features), np.array(labels)

# Extract manual features
train_manual, train_labels = extract_manual_features_from_folder("Split_IMG_CLASSES/train")
test_manual, test_labels = extract_manual_features_from_folder("Split_IMG_CLASSES/test")

# Normalize
scaler = StandardScaler()
train_manual_scaled = scaler.fit_transform(train_manual)
import pickle

# Save the fitted scaler
with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)
test_manual_scaled = scaler.transform(test_manual)

# Save
np.save("featuresets/train_manual_features.npy", train_manual_scaled)
np.save("featuresets/test_manual_features.npy", test_manual_scaled)
np.save("featuresets/train_manual_labels.npy", train_labels)
np.save("featuresets/test_manual_labels.npy", test_labels)

print("\nManual feature extraction complete and saved.")


7. Viral Infections: 100%|██████████| 200/200 [00:02<00:00, 77.36it/s]


Manual feature extraction complete and saved.


In [5]:
import os
import numpy as np
import cv2
from tqdm import tqdm
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenet_preprocess
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image

# Load MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')
model = Model(inputs=base_model.input, outputs=base_model.output)

def extract_deep_features(base_dir, model, preprocess_func):
    features = []
    labels = []
    class_names = sorted(os.listdir(base_dir))
    for label_index, class_name in enumerate(class_names):
        class_path = os.path.join(base_dir, class_name)
        for img_name in tqdm(os.listdir(class_path), desc=f"{class_name}"):
            img_path = os.path.join(class_path, img_name)
            img = cv2.imread(img_path)
            if img is None:
                continue
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (224, 224))
            img_array = image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array = preprocess_func(img_array)
            feat = model.predict(img_array, verbose=0)
            features.append(feat.flatten())
            labels.append(label_index)
    return np.array(features), np.array(labels)

# Extract MobileNetV2 features
mobilenet_train, mobilenet_train_labels = extract_deep_features("Split_IMG_CLASSES/train", model, mobilenet_preprocess)
mobilenet_test, mobilenet_test_labels = extract_deep_features("Split_IMG_CLASSES/test", model, mobilenet_preprocess)

# Save
np.save("featuresets/train_mobilenet_features.npy", mobilenet_train)
np.save("featuresets/test_mobilenet_features.npy", mobilenet_test)
np.save("featuresets/train_mobilenet_labels.npy", mobilenet_train_labels)
np.save("featuresets/test_mobilenet_labels.npy", mobilenet_test_labels)

print("\n MobileNetV2 features extracted and saved.")


C:\Users\RYZEN\AppData\Local\Temp\ipykernel_28332\2111761863.py:11: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')
7. Viral Infections: 100%|██████████| 200/200 [00:17<00:00, 11.31it/s]


 MobileNetV2 features extracted and saved.


In [6]:
import os
import numpy as np
import cv2
from tqdm import tqdm
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input as densenet_preprocess
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image

# Load DenseNet121 model (without top layer, with global average pooling)
base_model = DenseNet121(weights='imagenet', include_top=False, pooling='avg')
model = Model(inputs=base_model.input, outputs=base_model.output)

def extract_deep_features(base_dir, model, preprocess_func):
    features = []
    labels = []
    class_names = sorted(os.listdir(base_dir))
    for label_index, class_name in enumerate(class_names):
        class_path = os.path.join(base_dir, class_name)
        for img_name in tqdm(os.listdir(class_path), desc=f"{class_name}"):
            img_path = os.path.join(class_path, img_name)
            img = cv2.imread(img_path)
            if img is None:
                continue
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (224, 224))
            img_array = image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array = preprocess_func(img_array)
            feat = model.predict(img_array, verbose=0)
            features.append(feat.flatten())
            labels.append(label_index)
    return np.array(features), np.array(labels)

# Extract DenseNet features
densenet_train, densenet_train_labels = extract_deep_features("Split_IMG_CLASSES/train", model, densenet_preprocess)
densenet_test, densenet_test_labels = extract_deep_features("Split_IMG_CLASSES/test", model, densenet_preprocess)

# Save
np.save("featuresets/train_densenet_features.npy", densenet_train)
np.save("featuresets/test_densenet_features.npy", densenet_test)
np.save("featuresets/train_densenet_labels.npy", densenet_train_labels)
np.save("featuresets/test_densenet_labels.npy", densenet_test_labels)

print("\n DenseNet121 features extracted and saved.")


29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 18s 1us/step


7. Viral Infections: 100%|██████████| 200/200 [00:26<00:00,  7.49it/s]


 DenseNet121 features extracted and saved.
